In [1]:
import geopandas as gpd
import shapely.geometry
from shapely import wkt
import pandas as pd
import ast
from shapely.ops import unary_union
from shapely.geometry import MultiPolygon, Polygon
from shapely.validation import make_valid
import json
import os
import numpy as np

In [3]:
# Loading LR and ORD data with owner 
reamining_fields = pd.read_csv("/Users/smit0225/Documents/Oxford/Agriculture/Land_Registry/filtered_fields/new_defra_data/final_database/without cluster/LR_with_owner_unique/final_LR_with_&_without_crop_owner_not_in_country_remaining_ord.csv",index_col=False)
reamining_fields['geometry'] = reamining_fields['geometry'].apply(wkt.loads)
reamining_fields = gpd.GeoDataFrame(reamining_fields)
reamining_fields = reamining_fields.set_crs(27700, allow_override=True)
reamining_fields

/var/folders/px/16g77p5171d2dplxlk6vn8_h0000gr/T/ipykernel_5436/1986489310.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  reamining_fields2 = pd.read_csv("/Users/smit0225/Documents/Oxford/Agriculture/Land_Registry/filtered_fields/new_defra_data/final_database/without cluster/LR_with_owner_unique/final_LR_with_&_without_crop_owner_not_in_country_remaining_ord.csv",index_col=False)


,Unnamed: 0,fid,primary_ke,Proprietor Name (1),POLY_ID,Title Number,Tenure,Property Address,District,County,...,calculated,Area(hec),L_O_area_p,L_O_area_o,lucode,cromeid,Crom_area_lap,max_lucode_occurance,crop_maxA,geometry
0,0,osgb1000001762366543,3.085743e+10,GREAT TRODGERS FARM LIMITED,60659244,ESX398278,Freehold,"Great Trodgers Farm, Tunbridge Wells Road, May...",WEALDEN,EAST SUSSEX,...,24969.087296,2.497829,99.999988,2.497829,"['Spring Field beans', 'Spring Barley', 'Trees...","['RPA558166129719', 'RPA557986129684', 'RPA558...","[0.1403965153048041, 0.08899282497771051, 0.01...",4.0,Maize,"POLYGON ((558157.49 129727.72, 558163.63 12972..."
1,1,osgb5000005123860387,2.470077e+10,HARRON HOMES LIMITED,60826366,SYK450583,Freehold,"Land on the South side of Stairfoot Way, Barnsley",BARNSLEY,SOUTH YORKSHIRE,...,1003.932679,0.100489,75.994344,0.076366,"['Non agriculture land', 'Winter Field beans',...","['RPA437446404976', 'RPA437326404907', 'RPA437...","[0.0031428101030586686, 0.01130818524238348, 0...",4.0,Winter Field beans,"POLYGON ((437431.11 404970.82, 437429.61 40497..."
2,2,osgb5000005270489005,7.094326e+10,PROFUSION ENVIRONMENTAL LIMITED,57780663,DT419518,Freehold,"Woodville Farm, Green Lane, Stour Row, Shaftes...",DORSET,DORSET,...,9551.473291,0.956079,99.999999,0.956079,"['Grass', 'Lucerne', 'Grass']","['RPA382966120782', 'RPA382906120886', 'RPA382...","[0.24212375389768107, 0.23890851763903526, 0.1...",2.0,Grass,"POLYGON ((382925.78 120899.87, 382928.21 12089..."
3,3,osgb1000000034042722,7.555145e+10,TARMAC AGGREGATES LIMITED,36816591,HD457216,Freehold,"Land at Frogmore Park, Watton At Stone, Hertford",EAST HERTFORDSHIRE,HERTFORDSHIRE,...,12305.279240,1.231238,100.000000,1.231238,"['Grass', 'Grass', 'Grass', 'Trees and Scrubs,...","['RPA528586220375', 'RPA528646220340', 'RPA528...","[0.22234053338576673, 0.022492705575569042, 0....",3.0,Grass,"POLYGON ((528588.02 220453.32, 528588.12 22045..."
4,4,osgb1000001825221282,5.157762e+10,NORTHLANDS MEADOWS LIMITED,60929034,SY779134,Freehold,"land to the north and west of Northlands, Russ...",MOLE VALLEY,SURREY,...,26997.014070,2.701353,100.000000,2.701353,"['Fallow Land', 'Trees and Scrubs, short Woody...","['RPA522826140354', 'RPA522946140562', 'RPA522...","[0.3416074953672623, 0.27571462618789416, 0.25...",5.0,Maize,"POLYGON ((522959.1 140392.6, 522962.6 140384.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372149,372149,osgb1000000003110939,2.634559e+10,THE MORLEY AGRICULTURAL FOUNDATION,38628537,NK271362,Freehold,"Land on the east and west side of Church Road,...",SOUTH NORFOLK,NORFOLK,...,37469.342720,3.746761,100.000000,3.746761,"['Spring Barley', 'Spring Barley', 'Spring Bar...","['RPA604846301261', 'RPA604786301227', 'RPA604...","[0.40471312844699836, 0.41, 0.020605773017837586]",3.0,Spring Barley,"POLYGON ((604869.41 301373.1, 604877.77 301289..."
372150,372150,osgb5000005238862897,4.926800e+10,LINCOLNSHIRE COUNTY COUNCIL,59899585,LL130215,Freehold,land at Eastville,EAST LINDSEY,LINCOLNSHIRE,...,53449.992068,5.347619,95.230598,5.092570,"['Winter Oilseed', 'Winter Oilseed', 'Spring P...","['RPA541666359701', 'RPA542026359631', 'RPA541...","[0.03342195218255161, 0.027527387817009866, 0....",3.0,Beet,"POLYGON ((541987.53 359774.93, 541990.85 35974..."
372151,372151,osgb1000000403503544,2.965001e+10,WESTCOUNTRY MINERALS LIMITED,56569697,CL300686,Freehold,mines and minerals within and under the land s...,CORNWALL,CORNWALL,...,2329.195050,0.232855,100.000000,0.232855,NaN,NaN,NaN,NaN,NaN,"POLYGON ((173715.37 42242.01, 173721.07 42245...."
372152,372152,osgb1000002036750979,8.261703e+10,WHITWHAM FARM HOLDINGS LIMITED,20424977,ND144910,Freehold,"The Knarsdale Estate, Slaggyford",NORTHUMBERLAND,NORTHUMBERLAND,...,25906.334643,2.593122,100.000000,2.593122,"['Lucerne', 'Grass', 'Grass', 'Grass', 'Lucern...","['RPA367726555249', 'RPA367786555145', 'RPA367...","[0.29560231101463774, 0.41, 0.3646642076529415...",4.0,Grass,"POLYGON ((3

In [10]:
reamining_fields2[reamining_fields2['Proprietor Name (1)']=='07737281 LIMITED']['Company Registration No. (1)']

2408      07737281
4955      07737281
33712     07737281
54976     07737281
101370    07737281
165272    07737281
205226    07737281
205604    07737281
213817    07737281
300158    07737281
307166    07737281
323742    07737281
332752    07737281
335865    07737281
Name: Company Registration No. (1), dtype: object

In [27]:
#Cleaning
reamining_fields['Company Registration No. (1)'] = reamining_fields['Company Registration No. (1)'].apply(lambda x: x.lstrip('0') if isinstance(x, str) else x)
reamining_fields

,Unnamed: 0,fid,primary_ke,Proprietor Name (1),POLY_ID,Title Number,Tenure,Property Address,District,County,...,calculated,Area(hec),L_O_area_p,L_O_area_o,lucode,cromeid,Crom_area_lap,max_lucode_occurance,crop_maxA,geometry
0,0,osgb1000001762366543,3.085743e+10,GREAT TRODGERS FARM LIMITED,60659244,ESX398278,Freehold,"Great Trodgers Farm, Tunbridge Wells Road, May...",WEALDEN,EAST SUSSEX,...,24969.087296,2.497829,99.999988,2.497829,"['Spring Field beans', 'Spring Barley', 'Trees...","['RPA558166129719', 'RPA557986129684', 'RPA558...","[0.1403965153048041, 0.08899282497771051, 0.01...",4.0,Maize,"POLYGON ((558157.49 129727.72, 558163.63 12972..."
1,1,osgb5000005123860387,2.470077e+10,HARRON HOMES LIMITED,60826366,SYK450583,Freehold,"Land on the South side of Stairfoot Way, Barnsley",BARNSLEY,SOUTH YORKSHIRE,...,1003.932679,0.100489,75.994344,0.076366,"['Non agriculture land', 'Winter Field beans',...","['RPA437446404976', 'RPA437326404907', 'RPA437...","[0.0031428101030586686, 0.01130818524238348, 0...",4.0,Winter Field beans,"POLYGON ((437431.11 404970.82, 437429.61 40497..."
2,2,osgb5000005270489005,7.094326e+10,PROFUSION ENVIRONMENTAL LIMITED,57780663,DT419518,Freehold,"Woodville Farm, Green Lane, Stour Row, Shaftes...",DORSET,DORSET,...,9551.473291,0.956079,99.999999,0.956079,"['Grass', 'Lucerne', 'Grass']","['RPA382966120782', 'RPA382906120886', 'RPA382...","[0.24212375389768107, 0.23890851763903526, 0.1...",2.0,Grass,"POLYGON ((382925.78 120899.87, 382928.21 12089..."
3,3,osgb1000000034042722,7.555145e+10,TARMAC AGGREGATES LIMITED,36816591,HD457216,Freehold,"Land at Frogmore Park, Watton At Stone, Hertford",EAST HERTFORDSHIRE,HERTFORDSHIRE,...,12305.279240,1.231238,100.000000,1.231238,"['Grass', 'Grass', 'Grass', 'Trees and Scrubs,...","['RPA528586220375', 'RPA528646220340', 'RPA528...","[0.22234053338576673, 0.022492705575569042, 0....",3.0,Grass,"POLYGON ((528588.02 220453.32, 528588.12 22045..."
4,4,osgb1000001825221282,5.157762e+10,NORTHLANDS MEADOWS LIMITED,60929034,SY779134,Freehold,"land to the north and west of Northlands, Russ...",MOLE VALLEY,SURREY,...,26997.014070,2.701353,100.000000,2.701353,"['Fallow Land', 'Trees and Scrubs, short Woody...","['RPA522826140354', 'RPA522946140562', 'RPA522...","[0.3416074953672623, 0.27571462618789416, 0.25...",5.0,Maize,"POLYGON ((522959.1 140392.6, 522962.6 140384.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372149,372149,osgb1000000003110939,2.634559e+10,THE MORLEY AGRICULTURAL FOUNDATION,38628537,NK271362,Freehold,"Land on the east and west side of Church Road,...",SOUTH NORFOLK,NORFOLK,...,37469.342720,3.746761,100.000000,3.746761,"['Spring Barley', 'Spring Barley', 'Spring Bar...","['RPA604846301261', 'RPA604786301227', 'RPA604...","[0.40471312844699836, 0.41, 0.020605773017837586]",3.0,Spring Barley,"POLYGON ((604869.41 301373.1, 604877.77 301289..."
372150,372150,osgb5000005238862897,4.926800e+10,LINCOLNSHIRE COUNTY COUNCIL,59899585,LL130215,Freehold,land at Eastville,EAST LINDSEY,LINCOLNSHIRE,...,53449.992068,5.347619,95.230598,5.092570,"['Winter Oilseed', 'Winter Oilseed', 'Spring P...","['RPA541666359701', 'RPA542026359631', 'RPA541...","[0.03342195218255161, 0.027527387817009866, 0....",3.0,Beet,"POLYGON ((541987.53 359774.93, 541990.85 35974..."
372151,372151,osgb1000000403503544,2.965001e+10,WESTCOUNTRY MINERALS LIMITED,56569697,CL300686,Freehold,mines and minerals within and under the land s...,CORNWALL,CORNWALL,...,2329.195050,0.232855,100.000000,0.232855,NaN,NaN,NaN,NaN,NaN,"POLYGON ((173715.37 42242.01, 173721.07 42245...."
372152,372152,osgb1000002036750979,8.261703e+10,WHITWHAM FARM HOLDINGS LIMITED,20424977,ND144910,Freehold,"The Knarsdale Estate, Slaggyford",NORTHUMBERLAND,NORTHUMBERLAND,...,25906.334643,2.593122,100.000000,2.593122,"['Lucerne', 'Grass', 'Grass', 'Grass', 'Lucern...","['RPA367726555249', 'RPA367786555145', 'RPA367...","[0.29560231101463774, 0.41, 0.3646642076529415...",4.0,Grass,"POLYGON ((3

In [16]:
country_data = pd.read_csv("/Users/smit0225/Documents/Oxford/Agriculture/Land_Registry/filtered_fields/new_defra_data/LR/clean_LR_Ord/with_owner_name/all_together_with_owner_lR_ord_1_9M.csv",index_col=False)
country_data['geometry'] = country_data['geometry'].apply(wkt.loads)
country_data = gpd.GeoDataFrame(country_data)
country_data = country_data.set_crs(27700, allow_override=True)
country_data

/var/folders/px/16g77p5171d2dplxlk6vn8_h0000gr/T/ipykernel_5436/2958504139.py:1: DtypeWarning: Columns (20,26) have mixed types. Specify dtype option on import or set low_memory=False.
  country_data = pd.read_csv("/Users/smit0225/Documents/Oxford/Agriculture/Land_Registry/filtered_fields/new_defra_data/LR/clean_LR_Ord/with_owner_name/all_together_with_owner_lR_ord_1_9M.csv",index_col=False)


,Unnamed: 0,primary_ke,fid,Proprietor Name (1),POLY_ID,Title Number,Tenure,Property Address,District,County,...,Proprietor (2) Address (2),Proprietor (2) Address (3),Proprietor Name (3),Company Registration No. (3),Proprietorship Category (3),calculated,Area(hec),L_O_area_p,L_O_area_o,geometry
0,0,2.634665e+10,osgb1000000003011241,EAST DEREHAM TOWN COUNCIL,38347040,NK81594,Freehold,"land on the west and east side of Bush Lane, D...",BRECKLAND,NORFOLK,...,NaN,NaN,NaN,NaN,NaN,13125.209955,1.312503,99.982923,1.312279,"POLYGON ((601599.38 313747.33, 601602.83 31373..."
1,1,3.787208e+10,osgb1000000003026278,DNM REAL ESTATE LIMITED,38366591,NK141313,Freehold,"The Bungalow, Church Road, Bintree, Dereham (N...",BRECKLAND,NORFOLK,...,NaN,NaN,NaN,NaN,NaN,8217.485851,0.821736,99.999921,0.821736,"POLYGON ((601672.31 323598.49, 601705.931 3235..."
2,2,2.634910e+10,osgb1000000003036184,NORWICH DIOCESAN BOARD OF FINANCE LIMITED,38273875,NK361305,Freehold,"Land lying to the south of Slade Road, Bale, F...",NORTH NORFOLK,NORFOLK,...,NaN,NaN,NaN,NaN,NaN,9673.055984,0.967295,100.000000,0.967295,"POLYGON ((601186.99 336240.36, 601190.39 33623..."
3,3,7.657778e+10,osgb1000000003036192,ALBANWISE LIMITED,38345307,NK185270,Freehold,"Manor Farm, Bale, Fakenham and Valley Farm, Sh...",NORTH NORFOLK,NORFOLK,...,NaN,NaN,NaN,NaN,NaN,17481.884240,1.748166,99.999543,1.748158,"POLYGON ((601588.38 336304.5, 601590.26 336288..."
4,4,2.634719e+10,osgb1000000003056619,F. S. DANN & SON LIMITED,57732803,NK456365,Freehold,"Pound Farm, Pound Lane, North Tuddenham, Dereh...",BRECKLAND,NORFOLK,...,NaN,NaN,NaN,NaN,NaN,85210.586552,8.520810,99.999876,8.520799,"POLYGON ((603186.38 313798.9, 603195.26 313774..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499575,499575,8.455493e+10,osgb5000005296480555,VISTRY HOMES LIMITED,57714068,BD303398,Freehold,"Land lying to the north of Bromham Road, Bromh...",BEDFORD,BEDFORD,...,NaN,NaN,NaN,NaN,NaN,1789.584774,0.179088,92.997166,0.166547,"POLYGON ((502682.774 251025.831, 502679.297 25..."
499576,499576,8.441264e+10,osgb5000005296505513,HOMELAND SOLAR LIMITED,56158349,DT404749,Leasehold,"Land at Homeland Farm, Ringwood Road, Three Le...",DORSET,DORSET,...,NaN,NaN,NaN,NaN,NaN,1705.288894,0.170696,99.999888,0.170696,"POLYGON ((409395.922 106370.767, 409394.974 10..."
499577,499577,8.427530e+10,osgb5000005296512841,MILLS & REEVE TRUST CORPORATION LIMITED,31337615,LL261525,Freehold,"Land on the south side of Foxby Lane, Gainsbor...",WEST LINDSEY,LINCOLNSHIRE,...,NaN,NaN,NaN,NaN,NaN,92093.109120,9.216820,99.999982,9.216819,"POLYGON ((483092.65 388564.95, 483093.3 388564..."
499578,499578,8.443985e+10,osgb5000005296561122,BEAMES PROPERTY LIMITED,29081866,HP680508,Freehold,"LAND AT Pegham Coppice, Lavys Lane, Fareham",WINCHESTER,HAMPSHIRE,...,NaN,NaN,NaN,NaN,NaN,1893.876552,0.189560,31.906586,0.060482,"POLYGON ((454726.186 108871.46, 454718.52 1088..."


In [29]:
country_data['Country_side'] = 'Y'
country_data['Company Registration No. (1)'] = np.nan
country_data

,Unnamed: 0,primary_ke,fid,ORG_NAME,name_from_LR,calculated,CSREF,SUM_GIS_HA,C_O_A_prc,C_O_A_olap,Area_hc_st,Area(hec),crop_maxA,geometry,Country_side,Company Registration No. (1)
0,0,2.634564e+10,osgb1000000003003517,"BUNNING, G R",NaN,28093.336240,1631277,168.104473,99.999902,2.809310,168.074692,2.809313,Grass,"POLYGON ((600758.39 303651.94, 600758.3 303652...",Y,NaN
1,1,4.673184e+10,osgb1000000003003530,"BUNNING, G R",NaN,360963.355235,1631277,168.104473,99.999879,36.095737,168.074692,36.095781,Winter Wheat,"POLYGON ((602180.24 304707.52, 602180.25 30470...",Y,NaN
2,2,2.634566e+10,osgb1000000003003555,"BUNNING, G R",NaN,30532.341664,1631277,168.104473,99.999884,3.053211,168.074692,3.053215,Grass,"POLYGON ((600847.06 303783.18, 600839.97 30378...",Y,NaN
3,3,2.634560e+10,osgb1000000003003611,"BUNNING, G R",NaN,22832.199950,1631277,168.104473,99.778249,2.278123,168.074692,2.283186,Winter Wheat,"POLYGON ((601710.03 304187.93, 601708.3 304187...",Y,NaN
4,4,4.673118e+10,osgb1000000003004094,"BUNNING, G R",NaN,80832.077270,1631277,168.104473,98.842772,7.989590,168.074692,8.083131,Spring Barley,"POLYGON ((601024.96 304188.7, 601042.27 304255...",Y,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630641,630641,7.095609e+10,osgb1000000275265705,J & C Stanley,DORSET COUNCIL,3723.523956,1314565,35.759516,NaN,NaN,35.787889,0.372717,Lucerne,"POLYGON ((386063.1 120046.2, 386064.9 120043.2...",Y,NaN
630642,630642,7.771847e+10,osgb1000000181157909,RSPB,THE ROYAL SOCIETY FOR THE PROTECTION OF BIRDS,73260.405374,629476,303.349367,NaN,NaN,303.544627,7.332107,Spring Wheat,"POLYGON ((319901.711 561216.345, 319918.845 56...",Y,NaN
630643,630643,5.620509e+10,osgb5000005128032439,STAPLEFORD FARMS,STAPLEFORD ESTATE TRUSTEE ONE LIMITED,10893.446612,1053902,1036.834143,NaN,NaN,1037.490522,1.090229,Winter Wheat,"POLYGON ((483502.4 314551.35, 483501.77 314559...",Y,NaN
630644,630644,6.575373e+10,osgb1000002311434544,Linnell Bros Holdings Ltd,LINNELL BROTHERS HOLDINGS LIMITED,994.840795,1479763,27.259655,NaN,NaN,27.278330,0.099571,Lucerne,"POLYGON ((467438.91 245832.43, 467428.83 24582...",Y,NaN


In [26]:
country_data.columns

Index(['Owner Name', 'name_from_LR', 'Country_side',
       'Company Registration No. (1)', 'Area(hec)', 'crop_maxA', 'geometry'],
      dtype='object')

In [30]:
country_data = country_data[[ 'ORG_NAME', 'name_from_LR',
         'Country_side','Company Registration No. (1)', 'Area(hec)', 'crop_maxA', 'geometry']]
country_data = country_data.rename(columns={'ORG_NAME':'Owner Name'})
country_data

,Owner Name,name_from_LR,Country_side,Company Registration No. (1),Area(hec),crop_maxA,geometry
0,"BUNNING, G R",NaN,Y,NaN,2.809313,Grass,"POLYGON ((600758.39 303651.94, 600758.3 303652..."
1,"BUNNING, G R",NaN,Y,NaN,36.095781,Winter Wheat,"POLYGON ((602180.24 304707.52, 602180.25 30470..."
2,"BUNNING, G R",NaN,Y,NaN,3.053215,Grass,"POLYGON ((600847.06 303783.18, 600839.97 30378..."
3,"BUNNING, G R",NaN,Y,NaN,2.283186,Winter Wheat,"POLYGON ((601710.03 304187.93, 601708.3 304187..."
4,"BUNNING, G R",NaN,Y,NaN,8.083131,Spring Barley,"POLYGON ((601024.96 304188.7, 601042.27 304255..."
...,...,...,...,...,...,...,...
630641,J & C Stanley,DORSET COUNCIL,Y,NaN,0.372717,Lucerne,"POLYGON ((386063.1 120046.2, 386064.9 120043.2..."
630642,RSPB,THE ROYAL SOCIETY FOR THE PROTECTION OF BIRDS,Y,NaN,7.332107,Spring Wheat,"POLYGON ((319901.711 561216.345, 319918.845 56..."
630643,STAPLEFORD FARMS,STAPLEFORD ESTATE TRUSTEE ONE LIMITED,Y,NaN,1.090229,Winter Wheat,"POLYGON ((483502.4 314551.35, 483501.77 314559..."
630644,Linnell Bros Holdings Ltd,LINNELL BROTHERS HOLDINGS LIMITED,Y,NaN,0.099571,Lucerne,"POLYGON ((467438.91 245832.43, 467428.83 24582..."


In [31]:
reamining_fields = reamining_fields.rename(columns={'Proprietor Name (1)':"Owner Name"})
reamining_fields


,Unnamed: 0,fid,primary_ke,Owner Name,POLY_ID,Title Number,Tenure,Property Address,District,County,...,calculated,Area(hec),L_O_area_p,L_O_area_o,lucode,cromeid,Crom_area_lap,max_lucode_occurance,crop_maxA,geometry
0,0,osgb1000001762366543,3.085743e+10,GREAT TRODGERS FARM LIMITED,60659244,ESX398278,Freehold,"Great Trodgers Farm, Tunbridge Wells Road, May...",WEALDEN,EAST SUSSEX,...,24969.087296,2.497829,99.999988,2.497829,"['Spring Field beans', 'Spring Barley', 'Trees...","['RPA558166129719', 'RPA557986129684', 'RPA558...","[0.1403965153048041, 0.08899282497771051, 0.01...",4.0,Maize,"POLYGON ((558157.49 129727.72, 558163.63 12972..."
1,1,osgb5000005123860387,2.470077e+10,HARRON HOMES LIMITED,60826366,SYK450583,Freehold,"Land on the South side of Stairfoot Way, Barnsley",BARNSLEY,SOUTH YORKSHIRE,...,1003.932679,0.100489,75.994344,0.076366,"['Non agriculture land', 'Winter Field beans',...","['RPA437446404976', 'RPA437326404907', 'RPA437...","[0.0031428101030586686, 0.01130818524238348, 0...",4.0,Winter Field beans,"POLYGON ((437431.11 404970.82, 437429.61 40497..."
2,2,osgb5000005270489005,7.094326e+10,PROFUSION ENVIRONMENTAL LIMITED,57780663,DT419518,Freehold,"Woodville Farm, Green Lane, Stour Row, Shaftes...",DORSET,DORSET,...,9551.473291,0.956079,99.999999,0.956079,"['Grass', 'Lucerne', 'Grass']","['RPA382966120782', 'RPA382906120886', 'RPA382...","[0.24212375389768107, 0.23890851763903526, 0.1...",2.0,Grass,"POLYGON ((382925.78 120899.87, 382928.21 12089..."
3,3,osgb1000000034042722,7.555145e+10,TARMAC AGGREGATES LIMITED,36816591,HD457216,Freehold,"Land at Frogmore Park, Watton At Stone, Hertford",EAST HERTFORDSHIRE,HERTFORDSHIRE,...,12305.279240,1.231238,100.000000,1.231238,"['Grass', 'Grass', 'Grass', 'Trees and Scrubs,...","['RPA528586220375', 'RPA528646220340', 'RPA528...","[0.22234053338576673, 0.022492705575569042, 0....",3.0,Grass,"POLYGON ((528588.02 220453.32, 528588.12 22045..."
4,4,osgb1000001825221282,5.157762e+10,NORTHLANDS MEADOWS LIMITED,60929034,SY779134,Freehold,"land to the north and west of Northlands, Russ...",MOLE VALLEY,SURREY,...,26997.014070,2.701353,100.000000,2.701353,"['Fallow Land', 'Trees and Scrubs, short Woody...","['RPA522826140354', 'RPA522946140562', 'RPA522...","[0.3416074953672623, 0.27571462618789416, 0.25...",5.0,Maize,"POLYGON ((522959.1 140392.6, 522962.6 140384.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372149,372149,osgb1000000003110939,2.634559e+10,THE MORLEY AGRICULTURAL FOUNDATION,38628537,NK271362,Freehold,"Land on the east and west side of Church Road,...",SOUTH NORFOLK,NORFOLK,...,37469.342720,3.746761,100.000000,3.746761,"['Spring Barley', 'Spring Barley', 'Spring Bar...","['RPA604846301261', 'RPA604786301227', 'RPA604...","[0.40471312844699836, 0.41, 0.020605773017837586]",3.0,Spring Barley,"POLYGON ((604869.41 301373.1, 604877.77 301289..."
372150,372150,osgb5000005238862897,4.926800e+10,LINCOLNSHIRE COUNTY COUNCIL,59899585,LL130215,Freehold,land at Eastville,EAST LINDSEY,LINCOLNSHIRE,...,53449.992068,5.347619,95.230598,5.092570,"['Winter Oilseed', 'Winter Oilseed', 'Spring P...","['RPA541666359701', 'RPA542026359631', 'RPA541...","[0.03342195218255161, 0.027527387817009866, 0....",3.0,Beet,"POLYGON ((541987.53 359774.93, 541990.85 35974..."
372151,372151,osgb1000000403503544,2.965001e+10,WESTCOUNTRY MINERALS LIMITED,56569697,CL300686,Freehold,mines and minerals within and under the land s...,CORNWALL,CORNWALL,...,2329.195050,0.232855,100.000000,0.232855,NaN,NaN,NaN,NaN,NaN,"POLYGON ((173715.37 42242.01, 173721.07 42245...."
372152,372152,osgb1000002036750979,8.261703e+10,WHITWHAM FARM HOLDINGS LIMITED,20424977,ND144910,Freehold,"The Knarsdale Estate, Slaggyford",NORTHUMBERLAND,NORTHUMBERLAND,...,25906.334643,2.593122,100.000000,2.593122,"['Lucerne', 'Grass', 'Grass', 'Grass', 'Lucern...","['RPA367726555249', 'RPA367786555145', 'RPA367...","[0.29560231101463774, 0.41, 0.3646642076529415...",4.0,Grass,"POLYGON ((367705.14 

In [3]:
reamining_fields.columns

Index(['Unnamed: 0', 'ID', 'farm_name', 'New_full_a', 'Full Post', 'fid',
       'primary_ke', 'calculated', 'lucode', 'cromeid', 'Crom_area_lap',
       'max_lucode_occurance', 'Partial Po', 'Place type', 'Herbs & Or',
       'Other crop', 'Permanent', 'Temporary', 'Vegetables', 'crop_maxA',
       'Area(hec)', 'geometry'],
      dtype='object')

In [10]:
reamining_fields['Property Address']

0         Great Trodgers Farm, Tunbridge Wells Road, May...
1         Land on the South side of Stairfoot Way, Barnsley
2         Woodville Farm, Green Lane, Stour Row, Shaftes...
3          Land at Frogmore Park, Watton At Stone, Hertford
4         land to the north and west of Northlands, Russ...
                                ...                        
372149    Land on the east and west side of Church Road,...
372150                                    land at Eastville
372151    mines and minerals within and under the land s...
372152                     The Knarsdale Estate, Slaggyford
372153    land on the north west side of Bath Road, Brid...
Name: Property Address, Length: 372154, dtype: object

In [4]:
reamining_fields = reamining_fields[[ 'ID', 'farm_name', 'New_full_a', 'Full Post', 'fid',
       'primary_ke', 'calculated', 'lucode', 'cromeid', 'Crom_area_lap',
       'max_lucode_occurance', 'Partial Po', 'Place type', 'Herbs & Or',
       'Other crop', 'Permanent', 'Temporary', 'Vegetables', 'crop_maxA',
       'Area(hec)', 'geometry']]

reamining_fields

,ID,farm_name,New_full_a,Full Post,fid,primary_ke,calculated,lucode,cromeid,Crom_area_lap,...,Partial Po,Place type,Herbs & Or,Other crop,Permanent,Temporary,Vegetables,crop_maxA,Area(hec),geometry
0,FARM36749,MONEY HILL FARM,"MONEY HILL FARM, MONEY HILL FARM, DEOPHAM ROAD...",NR9 4NL,osgb1000000003000442,2.634589e+10,16021.021349,"['Grass', 'Lucerne']","['RPA601966301192', 'RPA601906301157']","[0.41, 0.16916221517850694]",...,4NL,NaN,NaN,NaN,NaN,NaN,NaN,Grass,1.602073,"POLYGON ((602029.29 301183.85, 602032.08 30117..."
1,FARM36749,MONEY HILL FARM,"MONEY HILL FARM, MONEY HILL FARM, DEOPHAM ROAD...",NR9 4NL,osgb1000000003000471,4.673119e+10,12044.726249,"['Winter Oilseed', 'Lucerne']","['RPA601786301157', 'RPA601906301157']","[0.09568223941103388, 0.08315439547114745]",...,4NL,NaN,NaN,NaN,NaN,NaN,NaN,Winter Oilseed,1.204452,"POLYGON ((601896.58 301175.91, 601898.06 30117..."
2,FARM17942,FROST ROW FARM,"UNNAMED ROAD, NORWICH, HINGHAM, UNITED KINGDOM",NaN,osgb1000000003000477,4.673142e+10,34519.895901,"['Winter Barley', 'Winter Oilseed', 'Grass', '...","['RPA601606301053', 'RPA601786301157', 'RPA601...","[0.02087073281263409, 0.06684406935046966, 0.4...",...,NaN,Poultry farm,NaN,NaN,NaN,NaN,NaN,Grass,3.451938,"POLYGON ((601736.271 301303.63, 601736.88 3012..."
3,FARM34777,MANSON GREEN FARM,"MANSON GREEN FARM, MANSON GREEN FARM, HIGHAM, ...",NR9 4PX,osgb1000000003000501,4.673227e+10,20208.105249,"['Trees and Scrubs, short Woody plants, hedger...",['RPA600826301919'],[0.015628439607740793],...,4PX,NaN,NaN,NaN,NaN,NaN,NaN,"Trees and Scrubs, short Woody plants, hedgerows",2.020795,"POLYGON ((600962.52 301904.47, 600947.613 3018..."
4,FARM09165,CHESTNUT FARM,"CHESTNUT FARM, CHESTNUT FARM, MONEY HILL, HING...",NR9 4NJ,osgb1000000003000519,2.634553e+10,249982.343872,"['Winter Wheat', 'Beet', 'Beet', 'Beet', 'Spri...","['RPA602326300846', 'RPA602086300568', 'RPA602...","[2.8437595928122132e-05, 0.13753200232964474, ...",...,4NJ,NaN,NaN,NaN,NaN,NaN,NaN,Beet,24.997758,"POLYGON ((601824.48 300517.47, 601816.41 30051..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1746451,FARM17523,FOUR ACRE FARM,"LONG LN, RINGWOOD, BH24 3BX, UNITED KINGDOM",BH24 3BX,osgb5000005296656689,8.442517e+10,6723.793296,['Non agriculture land'],['RPA415246103669'],[0.09313327829291068],...,3BX,Farm,NaN,NaN,NaN,NaN,NaN,Non agriculture land,0.673036,"POLYGON ((415334.176 103593.36, 415332.097 103..."
1746452,FARM44090,R & JM HARPER,"HEBBLETHWAITE MARTHWAITE, SEDBERGH, LA10 5HW, ...",LA10 5HW,osgb5000005296658301,8.416386e+10,8566.355360,['Winter Field beans'],['RPA362926491786'],[0.11840719119393733],...,5HW,Farm,NaN,NaN,NaN,NaN,NaN,Winter Field beans,0.857451,"POLYGON ((362853.536 491674.627, 362848.754 49..."
1746453,FARM00251,"3, PADGET RD","3 PADGET RD, RINGWOOD BH24 1TE, UK",BH24 1TE,osgb5000005296662587,8.442640e+10,3433.438230,"['Winter Field beans', 'Winter Field beans', '...","['RPA417046106856', 'RPA417046106925', 'RPA416...","[0.1701188734569757, 0.12064909397741597, 0.03...",...,1TE,NaN,NaN,NaN,NaN,NaN,NaN,Winter Field beans,0.343679,"POLYGON ((417020.618 106841.001, 417011.159 10..."
1746454,FARM36676,MOCKBEGGAR FARM,"4 CROSS LANES, MOCKBEGGAR, RINGWOOD BH24 3NQ, ...",BH24 3NQ,osgb5000005296665457,8.442656e+10,1197.340695,NaN,NaN,NaN,...,3NQ,Farm,NaN,NaN,NaN,NaN,NaN,NaN,0.119851,"POLYGON ((415122.192 109403.244, 415162.284 10..."


In [30]:
reamining_fields['Area(hec)'].sum()+country_data['Area(hec)'].sum()

4644177.110657909

In [11]:
country_data['Area(hec)'].sum()

3404907.8234036565

In [12]:
reamining_fields['Area(hec)'].sum()

1239269.2872542522

In [33]:
reamining_fields = pd.concat([reamining_fields,country_data])
reamining_fields = reamining_fields.reset_index(drop=True)
reamining_fields

,Owner Name,Company Registration No. (1),Proprietor (1) Address (1),District,County,Postcode,Proprietor Name (2),Area(hec),crop_maxA,geometry,name_from_LR,Country_side
0,GREAT TRODGERS FARM LIMITED,11679202,"Great Ailies Farm, Ailies Lane, East Hoathly, ...",WEALDEN,EAST SUSSEX,TN20 6NT,NaN,2.497829,Maize,"POLYGON ((558157.49 129727.72, 558163.63 12972...",NaN,NaN
1,HARRON HOMES LIMITED,3012678,"3 Colton Mill, Bullerthorpe Lane, Leeds LS15 9JN",BARNSLEY,SOUTH YORKSHIRE,NaN,NaN,0.100489,Winter Field beans,"POLYGON ((437431.11 404970.82, 437429.61 40497...",NaN,NaN
2,PROFUSION ENVIRONMENTAL LIMITED,7472762,"Woodville Farmhouse, Stour Row, Shaftesbury SP...",DORSET,DORSET,SP7 0QD,NaN,0.956079,Grass,"POLYGON ((382925.78 120899.87, 382928.21 12089...",NaN,NaN
3,TARMAC AGGREGATES LIMITED,297905,"Tarmac Legal Team, Ground Floor, T3 Trinity Pa...",EAST HERTFORDSHIRE,HERTFORDSHIRE,NaN,NaN,1.231238,Grass,"POLYGON ((528588.02 220453.32, 528588.12 22045...",NaN,NaN
4,NORTHLANDS MEADOWS LIMITED,13612011,"Northlands, Russ Hill, Charlwood, Horley RH6 0EL",MOLE VALLEY,SURREY,RH6 0EL,NaN,2.701353,Maize,"POLYGON ((522959.1 140392.6, 522962.6 140384.9...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1002795,J & C Stanley,NaN,NaN,NaN,NaN,NaN,NaN,0.372717,Lucerne,"POLYGON ((386063.1 120046.2, 386064.9 120043.2...",DORSET COUNCIL,Y
1002796,RSPB,NaN,NaN,NaN,NaN,NaN,NaN,7.332107,Spring Wheat,"POLYGON ((319901.711 561216.345, 319918.845 56...",THE ROYAL SOCIETY FOR THE PROTECTION OF BIRDS,Y
1002797,STAPLEFORD FARMS,NaN,NaN,NaN,NaN,NaN,NaN,1.090229,Winter Wheat,"POLYGON ((483502.4 314551.35, 483501.77 314559...",STAPLEFORD ESTATE TRUSTEE ONE LIMITED,Y
1002798,Linnell Bros Holdings Ltd,NaN,NaN,NaN,NaN,NaN,NaN,0.099571,Lucerne,"POLYGON ((467438.91 245832.43, 467428.83 24582...",LINNELL BROTHERS HOLDINGS LIMITED,Y


In [34]:
#final name cleaning code

import pandas as pd
import re

def conditional_upper(name):
    if name.isupper():
        return name
    else:
        return name.upper()

# Convert to uppercase


def replace_text(x):
    # Replace brackets with spaces
    pattern = r'[\[\](){}<>]'   
    return re.sub(pattern, ' ', x, flags=re.IGNORECASE)

def replace_hyph_text(x):
    # Replace hyphens between letters with spaces
    pattern = r'(?<=[a-zA-Z])-(?=[a-zA-Z])'   
    return re.sub(pattern, ' ', x, flags=re.IGNORECASE)

def replace_and(x):
    # Replace '&' with 'AND'
    return re.sub(r'&', 'AND', x)

def replace_LTD(x):
    # Replace 'LTD' with 'LIMITED'
    return re.sub(r'\bLTD\b', 'LIMITED', x)

def replace_property(x):
    # Replace 'PROPERTY' with 'PROPERTIES'
    return re.sub(r'PROPERTY', 'PROPERTIES', x)  

def remove_space(x):
    # Remove extra spaces
    x = re.sub(r'\s+', ' ', x)
    return x.strip()

def clean_commas(text):
    # Replace commas between words with spaces
    text = re.sub(r'(?<=[a-zA-Z]),(?=[a-zA-Z])', ' ', text)
    # Remove any trailing commas or trailing spaces
    text = re.sub(r',\s*$', '', text)
    # Remove leading or consecutive commas
    text = re.sub(r'(^,|,{2,})', '', text)
    # Remove extra spaces around commas
    text = re.sub(r'\s*,\s*', ' ', text)
    return text.strip()


def replace_special_characters_and_normalize(name):
    # Replace periods and hyphens with spaces
    replaced_chars = re.sub(r'[.-]', ' ', name)
    # Remove leading, trailing, and multiple spaces
    cleaned_name = re.sub(r'\s+', ' ', replaced_chars).strip()
    # Capitalize each word properly
    normalized_name = ' '.join(word.capitalize() for word in cleaned_name.split())
    return normalized_name

def remove_after_limited(x):
    pattern = r'(Limited).*'  # Match "Limited" and everything after
    return re.sub(pattern, r'\1', x, flags=re.IGNORECASE)


# Apply the functions to the 'company_na' column


#reamining_fields['farm_name'] = reamining_fields['farm_name'].apply(conditional_upper) 


#reamining_fields["farm_name"] = reamining_fields["farm_name"].apply(replace_text)
#reamining_fields["farm_name"] = reamining_fields["farm_namee"].apply(replace_hyph_text)
#reamining_fields["farm_name"] = reamining_fields["farm_name"].apply(replace_special_characters_and_normalize)
#reamining_fields["farm_name"] = reamining_fields["farm_name"].apply(remove_space)
#reamining_fields['farm_name'] = reamining_fields['farm_name'].apply(clean_commas)
#reamining_fields['farm_name'] = reamining_fields['farm_name'].apply(remove_after_limited)
#reamining_fields['farm_name'] = reamining_fields['farm_name'].str.replace(':', '').str.strip()

#reamining_fields['farm_name'] = reamining_fields['farm_name'].str.upper()
#reamining_fields['farm_name'] = reamining_fields['farm_name'].str.title()

reamining_fields['Owner Name'] = reamining_fields['Owner Name'].apply(conditional_upper) 
reamining_fields["Owner Name"] = reamining_fields["Owner Name"].apply(replace_text)
reamining_fields["Owner Name"] = reamining_fields["Owner Name"].apply(replace_hyph_text)
reamining_fields["Owner Name"] = reamining_fields["Owner Name"].apply(replace_LTD)
reamining_fields["Owner Name"] = reamining_fields["Owner Name"].apply(replace_and)
reamining_fields["Owner Name"] = reamining_fields["Owner Name"].apply(replace_property)
reamining_fields["Owner Name"] = reamining_fields["Owner Name"].apply(replace_special_characters_and_normalize)
reamining_fields["Owner Name"] = reamining_fields["Owner Name"].apply(remove_space)
reamining_fields['Owner Name'] = reamining_fields['Owner Name'].apply(clean_commas)
reamining_fields['Owner Name'] = reamining_fields['Owner Name'].apply(remove_after_limited)
reamining_fields['Owner Name'] = reamining_fields['Owner Name'].str.replace(':', '').str.strip()

reamining_fields['Owner Name'] = reamining_fields['Owner Name'].str.upper()
reamining_fields['Owner Name'] = reamining_fields['Owner Name'].str.title()



In [11]:
reamining_fields

,Owner Name,Company Registration No. (1),Proprietor (1) Address (1),District,County,Postcode,Proprietor Name (2),Area(hec),crop_maxA,geometry,name_from_LR,Country_side
0,Great Trodgers Farm Limited,11679202,"Great Ailies Farm, Ailies Lane, East Hoathly, ...",WEALDEN,EAST SUSSEX,TN20 6NT,NaN,2.497829,Maize,"POLYGON ((558157.49 129727.72, 558163.63 12972...",NaN,NaN
1,Harron Homes Limited,3012678,"3 Colton Mill, Bullerthorpe Lane, Leeds LS15 9JN",BARNSLEY,SOUTH YORKSHIRE,NaN,NaN,0.100489,Winter Field beans,"POLYGON ((437431.11 404970.82, 437429.61 40497...",NaN,NaN
2,Profusion Environmental Limited,7472762,"Woodville Farmhouse, Stour Row, Shaftesbury SP...",DORSET,DORSET,SP7 0QD,NaN,0.956079,Grass,"POLYGON ((382925.78 120899.87, 382928.21 12089...",NaN,NaN
3,Tarmac Aggregates Limited,297905,"Tarmac Legal Team, Ground Floor, T3 Trinity Pa...",EAST HERTFORDSHIRE,HERTFORDSHIRE,NaN,NaN,1.231238,Grass,"POLYGON ((528588.02 220453.32, 528588.12 22045...",NaN,NaN
4,Northlands Meadows Limited,13612011,"Northlands, Russ Hill, Charlwood, Horley RH6 0EL",MOLE VALLEY,SURREY,RH6 0EL,NaN,2.701353,Maize,"POLYGON ((522959.1 140392.6, 522962.6 140384.9...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1002795,J And C Stanley,NaN,NaN,NaN,NaN,NaN,NaN,0.372717,Lucerne,"POLYGON ((386063.1 120046.2, 386064.9 120043.2...",DORSET COUNCIL,Y
1002796,Rspb,NaN,NaN,NaN,NaN,NaN,NaN,7.332107,Spring Wheat,"POLYGON ((319901.711 561216.345, 319918.845 56...",THE ROYAL SOCIETY FOR THE PROTECTION OF BIRDS,Y
1002797,Stapleford Farms,NaN,NaN,NaN,NaN,NaN,NaN,1.090229,Winter Wheat,"POLYGON ((483502.4 314551.35, 483501.77 314559...",STAPLEFORD ESTATE TRUSTEE ONE LIMITED,Y
1002798,Linnell Bros Holdings Limited,NaN,NaN,NaN,NaN,NaN,NaN,0.099571,Lucerne,"POLYGON ((467438.91 245832.43, 467428.83 24582...",LINNELL BROTHERS HOLDINGS LIMITED,Y


In [35]:
re_id = pd.read_csv("/Users/smit0225/Documents/Oxford/Agriculture/Land_Registry/filtered_fields/new_defra_data/final_database/with_cluster/owner/base_owner/registration_no_name.csv")
re_id

,Company Registration No. (1),Owner Name
0,8496160,Emily Estate Uk Limited
1,(CIO REGN NO 1190035),Belmont Abbey General Cio
2,(OC304961),Loyton Llp
3,10000573,Trillium Care Limited
4,10000671,Fidelis Legal Services Limited
...,...,...
23435,SO305773,Gresham House Forest Growth And Sustainability Lp
23436,SO307143,Yorkshire Dales Ffc Llp
23437,ZC000145,Wesleyan Assurance Society
23438,ZC000159,Messrs Hoare Trustees


In [36]:
result_dict = dict(zip(re_id['Company Registration No. (1)'], re_id['Owner Name']))


In [37]:
reamining_fields['Owner Name'] = reamining_fields['Company Registration No. (1)'].map(result_dict).fillna(reamining_fields['Owner Name'])

reamining_fields

,Owner Name,Company Registration No. (1),Proprietor (1) Address (1),District,County,Postcode,Proprietor Name (2),Area(hec),crop_maxA,geometry,name_from_LR,Country_side
0,Great Trodgers Farm Limited,11679202,"Great Ailies Farm, Ailies Lane, East Hoathly, ...",WEALDEN,EAST SUSSEX,TN20 6NT,NaN,2.497829,Maize,"POLYGON ((558157.49 129727.72, 558163.63 12972...",NaN,NaN
1,Harron Homes Limited,3012678,"3 Colton Mill, Bullerthorpe Lane, Leeds LS15 9JN",BARNSLEY,SOUTH YORKSHIRE,NaN,NaN,0.100489,Winter Field beans,"POLYGON ((437431.11 404970.82, 437429.61 40497...",NaN,NaN
2,Profusion Environmental Limited,7472762,"Woodville Farmhouse, Stour Row, Shaftesbury SP...",DORSET,DORSET,SP7 0QD,NaN,0.956079,Grass,"POLYGON ((382925.78 120899.87, 382928.21 12089...",NaN,NaN
3,Tarmac Aggregates Limited,297905,"Tarmac Legal Team, Ground Floor, T3 Trinity Pa...",EAST HERTFORDSHIRE,HERTFORDSHIRE,NaN,NaN,1.231238,Grass,"POLYGON ((528588.02 220453.32, 528588.12 22045...",NaN,NaN
4,Northlands Meadows Limited,13612011,"Northlands, Russ Hill, Charlwood, Horley RH6 0EL",MOLE VALLEY,SURREY,RH6 0EL,NaN,2.701353,Maize,"POLYGON ((522959.1 140392.6, 522962.6 140384.9...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1002795,J And C Stanley,NaN,NaN,NaN,NaN,NaN,NaN,0.372717,Lucerne,"POLYGON ((386063.1 120046.2, 386064.9 120043.2...",DORSET COUNCIL,Y
1002796,Rspb,NaN,NaN,NaN,NaN,NaN,NaN,7.332107,Spring Wheat,"POLYGON ((319901.711 561216.345, 319918.845 56...",THE ROYAL SOCIETY FOR THE PROTECTION OF BIRDS,Y
1002797,Stapleford Farms,NaN,NaN,NaN,NaN,NaN,NaN,1.090229,Winter Wheat,"POLYGON ((483502.4 314551.35, 483501.77 314559...",STAPLEFORD ESTATE TRUSTEE ONE LIMITED,Y
1002798,Linnell Bros Holdings Limited,NaN,NaN,NaN,NaN,NaN,NaN,0.099571,Lucerne,"POLYGON ((467438.91 245832.43, 467428.83 24582...",LINNELL BROTHERS HOLDINGS LIMITED,Y


In [56]:
data1 = reamining_fields[~reamining_fields['crop_maxA'].isna()]
data2 = reamining_fields[reamining_fields['crop_maxA'].isna()]

In [42]:


def combine_sets(series):
    combined = set()
    for subset in series:
        combined.update(subset)
    return combined

f = { 'Company Registration No. (1)':lambda x: ((x.tolist())[0]),
         'Proprietor (1) Address (1)':lambda x: ((x.tolist())[0]),
        'District':lambda x: ((x.tolist())[0]), 
    'County':lambda x: ((x.tolist())[0]), 
    'Postcode':lambda x: ((x.tolist())[0]), 
     'Proprietor Name (2)':lambda x: list(set(x.tolist()))[0],
     'Country_side':lambda x: x.dropna().iloc[0] if not x.dropna().empty else np.nan,
  
     'name_from_LR':lambda x: x.dropna().iloc[0] if not x.dropna().empty else np.nan,
       'Area(hec)':lambda x: sum(x.tolist()),
       
      }#'crop_maxA':lambda x: list(set(x.dropna())) if not x.dropna().empty else np.nan,
    

z_final1 = (data1.groupby(['Owner Name','crop_maxA'])
       .agg(f)
       .reset_index())
temp = data1.groupby(['Owner Name','crop_maxA'])['geometry'].apply(list).reset_index(name='geometry')
z_final1['geometry'] = temp['geometry'].apply(lambda x: unary_union(x))

z_final1



,Owner Name,crop_maxA,Company Registration No. (1),Proprietor (1) Address (1),District,County,Postcode,Proprietor Name (2),Country_side,name_from_LR,Area(hec),geometry
0,001Hardy Limited,Grass,11046698,"Willow Farm, Newark Road, Hawton, Newark NG24 3RR",NEWARK AND SHERWOOD,NOTTINGHAMSHIRE,NaN,NaN,NaN,NaN,4.956177,MULTIPOLYGON (((478553.84999990463 351114.3099...
1,001Hardy Limited,Spring Field beans,11046698,"Willow Farm, Newark Road, Hawton, Newark NG24 3RR",NEWARK AND SHERWOOD,NOTTINGHAMSHIRE,NaN,NaN,NaN,NaN,0.067729,"POLYGON ((478898.8899998665 351501.9800000191,..."
2,001Hardy Limited,Spring Oats,11046698,"Willow Farm, Newark Road, Hawton, Newark NG24 3RR",NEWARK AND SHERWOOD,NOTTINGHAMSHIRE,NaN,NaN,NaN,NaN,2.664547,POLYGON ((478549.19899988174 351342.2750000953...
3,07737281 Limited,Grass,7737281,"Russell House, Oxford Road, Bournemouth BH8 8EX",WILTSHIRE,WILTSHIRE,SP7 8QG,NaN,NaN,NaN,10.179522,MULTIPOLYGON (((387523.45200014114 123177.0939...
4,07737281 Limited,Lucerne,7737281,"Russell House, Oxford Road, Bournemouth BH8 8EX",WILTSHIRE,WILTSHIRE,SP7 8QG,NaN,NaN,NaN,0.176598,POLYGON ((387643.40399980545 123196.3850002288...
...,...,...,...,...,...,...,...,...,...,...,...,...
62072,Zyox Limited,Spring Barley,2270075,"Swallow House, Shilton Industrial Estate, Shil...",CHERWELL,OXFORDSHIRE,NaN,NaN,NaN,NaN,32.830963,MULTIPOLYGON (((458555.90000009537 212182.0500...
62073,Zyox Limited,Spring Wheat,2270075,"Swallow House, Shilton Industrial Estate, Shil...",CHERWELL,OXFORDSHIRE,NaN,NaN,NaN,NaN,9.682011,MULTIPOLYGON (((458690.28999996185 212494.2300...
62074,Zyox Limited,Winter Wheat,2270075,"Swallow House, Shilton Industrial Estate, Shil...",CHERWELL,OXFORDSHIRE,NaN,NaN,NaN,NaN,29.356238,MULTIPOLYGON (((452810.7300000191 209878.07999...
62075,Zziggzzagg Limited,Non agriculture land,12296846,"Bunters Gait, Nunnery Green, Wickhambrook, New...",WEST SUFFOLK,SUFFOLK,CB8 8XT,NaN,NaN,NaN,0.435946,POLYGON ((574405.6779999733 255835.97499990463...


In [43]:
z_final1[z_final1['Owner Name']=='Emily Estate Uk Limited']

,Owner Name,crop_maxA,Company Registration No. (1),Proprietor (1) Address (1),District,County,Postcode,Proprietor Name (2),Country_side,name_from_LR,Area(hec),geometry
16821,Emily Estate Uk Limited,Clover,8496160,"Hadspen House, Hadspen, Castle Cary BA7 7NG",SOMERSET,SOMERSET,BA7 7BA,NaN,NaN,NaN,10.061552,MULTIPOLYGON (((364990.44999980927 130402.2399...
16822,Emily Estate Uk Limited,Fallow Land,8496160,"Estate Office, Hadspen House, Hadspen, Castle ...",SOMERSET,SOMERSET,BA7 7NG,NaN,NaN,NaN,9.601628,"POLYGON ((366127.992000103 130543.23500013351,..."
16823,Emily Estate Uk Limited,Grass,8496160,"Hadspen House, Hadspen, Castle Cary BA7 7NG",SOMERSET,SOMERSET,BA9 8DG,NaN,NaN,NaN,185.643142,MULTIPOLYGON (((231070.23000001907 67309.90000...
16824,Emily Estate Uk Limited,HEAT,8496160,"Estate Office, Hadspen House, Hadspen, Castle ...",SOMERSET,SOMERSET,BA7 7NG,NaN,NaN,NaN,0.439996,"POLYGON ((366625.40000009537 130601, 366622.90..."
16825,Emily Estate Uk Limited,Lucerne,8496160,"Estate Office, Hadspen House, Hadspen, Castle ...",SOMERSET,SOMERSET,BA9 8DJ,NaN,NaN,NaN,87.343419,MULTIPOLYGON (((365813.67999982834 129147.7300...
16826,Emily Estate Uk Limited,Maize,10518057,"Estate Office, Hadspen House, Hadspen, Castle ...",SOMERSET,SOMERSET,BA22 7BX,NaN,NaN,NaN,59.430850,MULTIPOLYGON (((231048.5300002098 67248.199999...
16827,Emily Estate Uk Limited,Spring Barley,8496160,"Hadspen House, Hadspen, Castle Cary BA7 7NG",SOMERSET,SOMERSET,NaN,NaN,NaN,NaN,9.789297,MULTIPOLYGON (((366231.9400000572 129860.51000...
16828,Emily Estate Uk Limited,Spring Field beans,8496160,"Estate Office, Hadspen House, Hadspen, Castle ...",SOMERSET,SOMERSET,BA7 7NF,NaN,NaN,NaN,0.092113,"POLYGON ((366835.80000019073 130127, 366833.90..."
16829,Emily Estate Uk Limited,Spring Oats,8496160,"Hadspen House, Hadspen, Castle Cary BA7 7NG",SOMERSET,SOMERSET,BA7 7BA,NaN,NaN,NaN,0.801743,POLYGON ((364051.6659998894 131140.01700019836...
16830,Emily Estate Uk Limited,"Trees and Scrubs, short Woody plants, hedgerows",8496160,"Estate Office, Hadspen House, Hadspen, Castle ...",SOMERSET,SOMERSET,BA9 8DJ,NaN,NaN,NaN,5.469806,MULTIPOLYGON (((231007.41400003433 67056.90299...


In [44]:
# Merging only based on Owner name

def combine_sets(series):
    combined = set()
    for subset in series:
        combined.update(subset)
    return combined

f = {'Company Registration No. (1)':lambda x: list(set(x.dropna()))[0] if not x.dropna().empty else np.nan,
         'Proprietor (1) Address (1)':lambda x: ((x.tolist())[0]),
        'District':lambda x: ((x.tolist())[0]), 
    'County':lambda x: ((x.tolist())[0]), 
    'Postcode':lambda x: ((x.tolist())[0]), 
     'Proprietor Name (2)':lambda x: list(set(x.tolist()))[0],
     'Country_side':lambda x: x.dropna().iloc[0] if not x.dropna().empty else np.nan,
  
     'name_from_LR':lambda x: x.dropna().iloc[0] if not x.dropna().empty else np.nan,
       'Area(hec)':lambda x: sum(x.tolist()),
       'crop_maxA':lambda x: list(set(x.dropna())) if not x.dropna().empty else '',
      }#
    

z_final2 = (data2.groupby(['Owner Name'])
       .agg(f)
       .reset_index())

temp = data2.groupby(['Owner Name'])['geometry'].apply(list).reset_index(name='geometry')
z_final2['geometry'] = temp['geometry'].apply(lambda x: unary_union(x))
z_final2



,Owner Name,Company Registration No. (1),Proprietor (1) Address (1),District,County,Postcode,Proprietor Name (2),Country_side,name_from_LR,Area(hec),crop_maxA,geometry
0,12 Properties Fe Limited,9351326,"3 Brook Business Centre, Cowley Mill Road, Cow...",CUMBERLAND,CUMBERLAND,NaN,NaN,NaN,NaN,19.423233,,MULTIPOLYGON (((299270.90000009537 522757.2899...
1,1St Choice Animals Limited,5197807,"2 Wheatmere Drove, Low Fulney, Spalding PE12 6NA",SOUTH HOLLAND,LINCOLNSHIRE,PE12 6NA,NaN,NaN,NaN,0.199005,,MULTIPOLYGON (((526972.1999998093 322474.76000...
2,2 In Tents Marquee Hire Limited,8963280,"55 East Street, Faversham ME13 8AF",ASHFORD,KENT,TN26 1NL,NaN,NaN,NaN,1.077288,,"POLYGON ((599449.25 137028.9699997902, 599454...."
3,22Hd Limited,12700118,"51 Abbots Road, Abbots Langley, WD5 0BJ",HARBOROUGH,LEICESTERSHIRE,LE2 4RG,NaN,NaN,NaN,0.091325,,POLYGON ((464172.96000003815 298399.5999999046...
4,25 Kgf Limited,9458831,"Braecroft, Upper Oddington, Moreton-In-Marsh G...",COTSWOLD,GLOUCESTERSHIRE,NaN,NaN,NaN,NaN,0.060833,,POLYGON ((419924.94999980927 227736.4499998092...
...,...,...,...,...,...,...,...,...,...,...,...,...
5254,Zero Three Care Homes Llp,OC308106,"Suite 3, The Exchange Court, London Road, Feer...",BRAINTREE,ESSEX,CM8 3PJ,NaN,NaN,NaN,0.432496,,"POLYGON ((581294.1799998283 219922.2199997902,..."
5255,Zero Two Uk Holdings Limited,8501233,"Monometer Holdings Ltd, Monometer House, Recto...",BRAINTREE,ESSEX,CO9 4HP,NaN,NaN,NaN,0.855537,,POLYGON ((576604.1599998474 237131.53999996185...
5256,Zigcam Pic Limited,6538141,"Zigcam Film & Television Unit 10, Greenwich Ce...",NORTH NORFOLK,NORFOLK,NR25 6LQ,NaN,NaN,NaN,0.340857,,"POLYGON ((611942.0349998474 336302.7680001259,..."
5257,Ziggus Holdings Limited,3557956,"Sampford Barton Farm, Sampford Spiney, Yelvert...",WEST DEVON,DEVON,PL20 6LD,NaN,NaN,NaN,3.188201,,MULTIPOLYGON (((252669.81699991226 71236.45499...


In [77]:
owner_ids = pd.read_csv("/Users/smit0225/Documents/Oxford/Agriculture/Land_Registry/filtered_fields/new_defra_data/final_database/with_cluster/owner/base_owner/owner_id_owner_name_mapping.csv", index_col=False)
owner_ids

,Owner Name,OWNER_ID
0,001Hardy Limited,OWNER00001
1,07737281 Limited,OWNER00002
2,1 Grosvenor Holdings Limited,OWNER00003
3,106 Properties Group Limited,OWNER00004
4,112680647,OWNER00005
...,...,...
58000,Zurich Developments Limited,OWNER58001
58001,Zym Capital Limited,OWNER58002
58002,Zymshire Limited,OWNER58003
58003,Zyox Limited,OWNER58004


In [79]:
idx_dict = dict(zip(owner_ids['Owner Name'], owner_ids['OWNER_ID']))


In [74]:
idx_dict = dict(zip(z_final1['Owner Name'], z_final1['OWNER_ID']))

df = pd.DataFrame(list(idx_dict.items()), columns=['Owner Name', 'OWNER_ID'])

# Save DataFrame to CSV
df.to_csv('/Users/smit0225/Documents/Oxford/Agriculture/Land_Registry/filtered_fields/new_defra_data/final_database/with_cluster/owner/base_owner/owner_id_owner_name_mapping.xlsx', index=False)


In [109]:
idx_dict2 = dict(zip(owner_ids['Company Registration No. (1)'], owner_ids['OWNER_ID']))


In [80]:
z_final_crop['OWNER_ID'] = z_final2['Owner Name'].map(idx_dict)
z_final_crop

,Owner Name,crop_maxA,Company Registration No. (1),Proprietor (1) Address (1),District,County,Postcode,Proprietor Name (2),Country_side,name_from_LR,Area(hec),geometry,OWNER_ID
0,001Hardy Limited,Grass,11046698,"Willow Farm, Newark Road, Hawton, Newark NG24 3RR",NEWARK AND SHERWOOD,NOTTINGHAMSHIRE,NaN,NaN,NaN,NaN,4.956177,MULTIPOLYGON (((478547.67999982834 351117.7399...,OWNER00001
1,001Hardy Limited,Spring Field beans,11046698,"Willow Farm, Newark Road, Hawton, Newark NG24 3RR",NEWARK AND SHERWOOD,NOTTINGHAMSHIRE,NaN,NaN,NaN,NaN,0.067729,"POLYGON ((478898.8899998665 351501.9800000191,...",OWNER00001
2,001Hardy Limited,Spring Oats,11046698,"Willow Farm, Newark Road, Hawton, Newark NG24 3RR",NEWARK AND SHERWOOD,NOTTINGHAMSHIRE,NaN,NaN,NaN,NaN,2.664547,POLYGON ((478549.19899988174 351342.2750000953...,OWNER00001
3,07737281 Limited,Grass,7737281,"Russell House, Oxford Road, Bournemouth BH8 8EX",WILTSHIRE,WILTSHIRE,SP7 8QG,NaN,NaN,NaN,10.179522,MULTIPOLYGON (((387712.7940001488 123142.09499...,OWNER00002
4,07737281 Limited,Lucerne,7737281,"Russell House, Oxford Road, Bournemouth BH8 8EX",WILTSHIRE,WILTSHIRE,SP7 8QG,NaN,NaN,NaN,0.176598,POLYGON ((387643.40399980545 123196.3850002288...,OWNER00002
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251711,Zero Two Uk Holdings Limited,,8501233,"Monometer Holdings Ltd, Monometer House, Recto...",BRAINTREE,ESSEX,CO9 4HP,NaN,NaN,NaN,0.855537,POLYGON ((576604.1599998474 237131.53999996185...,OWNER57984
251712,Zetland Country Limited,,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,1.430863,MULTIPOLYGON (((449899.0300002098 487418.69000...,OWNER57985
251713,Zigcam Pic Limited,,6538141,"Zigcam Film & Television Unit 10, Greenwich Ce...",NORTH NORFOLK,NORFOLK,NR25 6LQ,NaN,NaN,NaN,0.340857,"POLYGON ((611942.0349998474 336302.7680001259,...",OWNER57988
251714,Ziggus Holdings Limited,,3557956,"Sampford Barton Farm, Sampford Spiney, Yelvert...",WEST DEVON,DEVON,PL20 6LD,NaN,NaN,NaN,3.188201,MULTIPOLYGON (((252676.26000022888 71232.73000...,OWNER57989


In [87]:
z_final_crop = z_final_crop[ ['OWNER_ID', 'Owner Name', 'crop_maxA', 'Company Registration No. (1)',
       'Proprietor (1) Address (1)', 'District', 'County', 'Postcode',
       'Proprietor Name (2)', 'Country_side', 'name_from_LR', 'Area(hec)',
       'geometry']]
z_final_crop

,OWNER_ID,Owner Name,crop_maxA,Company Registration No. (1),Proprietor (1) Address (1),District,County,Postcode,Proprietor Name (2),Country_side,name_from_LR,Area(hec),geometry
0,OWNER00001,001Hardy Limited,Grass,11046698,"Willow Farm, Newark Road, Hawton, Newark NG24 3RR",NEWARK AND SHERWOOD,NOTTINGHAMSHIRE,NaN,NaN,NaN,NaN,4.956177,MULTIPOLYGON (((478547.67999982834 351117.7399...
1,OWNER00001,001Hardy Limited,Spring Field beans,11046698,"Willow Farm, Newark Road, Hawton, Newark NG24 3RR",NEWARK AND SHERWOOD,NOTTINGHAMSHIRE,NaN,NaN,NaN,NaN,0.067729,"POLYGON ((478898.8899998665 351501.9800000191,..."
2,OWNER00001,001Hardy Limited,Spring Oats,11046698,"Willow Farm, Newark Road, Hawton, Newark NG24 3RR",NEWARK AND SHERWOOD,NOTTINGHAMSHIRE,NaN,NaN,NaN,NaN,2.664547,POLYGON ((478549.19899988174 351342.2750000953...
3,OWNER00002,07737281 Limited,Grass,7737281,"Russell House, Oxford Road, Bournemouth BH8 8EX",WILTSHIRE,WILTSHIRE,SP7 8QG,NaN,NaN,NaN,10.179522,MULTIPOLYGON (((387712.7940001488 123142.09499...
4,OWNER00002,07737281 Limited,Lucerne,7737281,"Russell House, Oxford Road, Bournemouth BH8 8EX",WILTSHIRE,WILTSHIRE,SP7 8QG,NaN,NaN,NaN,0.176598,POLYGON ((387643.40399980545 123196.3850002288...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251711,OWNER57984,Zero Two Uk Holdings Limited,,8501233,"Monometer Holdings Ltd, Monometer House, Recto...",BRAINTREE,ESSEX,CO9 4HP,NaN,NaN,NaN,0.855537,POLYGON ((576604.1599998474 237131.53999996185...
251712,OWNER57985,Zetland Country Limited,,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,1.430863,MULTIPOLYGON (((449899.0300002098 487418.69000...
251713,OWNER57988,Zigcam Pic Limited,,6538141,"Zigcam Film & Television Unit 10, Greenwich Ce...",NORTH NORFOLK,NORFOLK,NR25 6LQ,NaN,NaN,NaN,0.340857,"POLYGON ((611942.0349998474 336302.7680001259,..."
251714,OWNER57989,Ziggus Holdings Limited,,3557956,"Sampford Barton Farm, Sampford Spiney, Yelvert...",WEST DEVON,DEVON,PL20 6LD,NaN,NaN,NaN,3.188201,MULTIPOLYGON (((252676.26000022888 71232.73000...


In [83]:
z_final = z_final.reset_index(drop=True)
z_final = gpd.GeoDataFrame(z_final)
z_final = z_final.set_crs('epsg:27700')
z_final.to_csv("/Users/smit0225/Documents/Oxford/Agriculture/Land_Registry/filtered_fields/new_defra_data/final_database/with_cluster/farm_voronoi/farm_crop_wise/farm_crop_wise_with_geometry.csv")